```
 DPI/
   ├── dpi/
   │   ├── __init__.py
   │   ├── dilemma.py  # implementa la lógica del DP jugado, acciones posibles, matriz de pagos, etc
   │   ├── player.py  # implementa la definición de "Jugador" y en particular, la estrategia
   │   ├── game.py  # implementa la dinámica de un juego cara a cara entre dos jugadores
   │   ├── tournament.py  # implementa la dinámica del torneo general
   │   └── evolution.py  # implementa la dinámica del torneo evolutivo
   ├── main.py # main del proyecto - si quieres, usa varios 'mains' para testear los distintos módulos
   └──...
```

In [1]:
from __future__ import annotations

from abc import ABC, abstractmethod
import numpy as np
import copy
import math
import matplotlib.pyplot as plt
import itertools
import random

## **CLASE DILEMA**

In [3]:
class Dilemma:

    def __init__(self, cc: float, cd: float, dc: float, dd: float):
        """
        Represents a 2x2 symmetric dilemma.

        Parameters:
            - cc (float): payoff for mutual cooperation
            - cd (float): payoff when one cooperates, but the opponent defects
            - dc (float): payoff when one defects, and the opponent cooperates
            - dd (float): payoff for mutual defection
        """
        self.cc=cc
        self.cd=cd
        self.dc=dc
        self.dd=dd
        self.matrix=np.array([[self.cc,self.cd],[self.dc,self.dd]])


    @property
    @abstractmethod
    def payoff_matrix(self) -> np.ndarray:
        """
        Returns:
            - 2x2 np array of the matrix
        """
        return self.matrix


    @abstractmethod
    def evaluate_result(self, a_1: int, a_2: int) -> tuple[float, float]:
        """
        Given two actions, returns the payoffs of the two players.

        Parameters:
            - a_1 (int): action of player 1 ('C' or 'D', i.e. '1' or '0')
            - a_2 (int): action of player 2 ('C' or 'D', i.e. '1' or '0')

        Returns:
            - tuple of two floats, being the first and second values the payoff
            for the first and second player, respectively.
        """
        return int(self.matrix[a_1,a_2]),int(self.matrix[a_2,a_1])

In [ ]:
C = 0
D = 1

In [21]:
# Prints all possible outcomes of the PD
pd = Dilemma(2, -1, 3, 0)
posible_actions = (C, D)
for a1, a2 in itertools.product(posible_actions, repeat=2):
    print(f"{(a1, a2)} -> {pd.evaluate_result(a1, a2)}")

(0, 0) -> (2, 2)
(0, 1) -> (-1, 3)
(1, 0) -> (3, -1)
(1, 1) -> (0, 0)


## **CLASE PLAYER**


In [42]:
class Player(ABC):

    @abstractmethod
    def __init__(self, dilemma: Dilemma, name: str = ""):
        """
        Abstract class that represents a generic player

        Parameters:
            - name (str): the name of the strategy
            - dilemma (Dilemma): the dilemma that this player will play
        """

        self.name = name
        self.dilemma = dilemma

        self.history  = []  # This is the main variable of this class. It is
                            # intended to store all the history of actions
                            # performed by this player.
                            # Example: [C, C, D, D, D] <- So far, the
                            # interaction lasts five rounds. In the first one,
                            # this player cooperated. In the second, he also
                            # cooperated. In the third, he defected. Etc.
        self.trust=True
        self.iteration=0


    @abstractmethod
    def strategy(self, opponent: Player) -> int:
        """
        Main call of the class. Gives the action for the following round of the
        interaction, based on the history

        Parameters:
            - opponent (Player): is another instance of Player.

        Results:
            - An integer representing Cooperation (C=0) or Defection (D=1)
        """
        pass


    def compute_scores(self, opponent: Player) -> tuple[float, float]:
        """
        Compute the scores for a given opponent

        Parameters:
            - opponent (Player): is another instance of Player.

        Results:
            - A tuple of two floats, where the first value is the current
            player's payoff, and the second value is the opponent's payoff.
        """
        other_player_actions=opponent.history
        results = np.array([self.dilemma.evaluate_result(action1,action2) for action1,action2 in zip(self.history,other_player_actions)])
        current_player_payoff=results[:,0].sum()
        opponent_player_payoff=results[:,1].sum()
        return int(current_player_payoff), int(opponent_player_payoff)


    def clean_history(self):
        """Resets the history of the current player"""
        self.history = []
        self.iteration=0
        self.trust=True


# Las 5 estrategias básicas del juego de Nicky Case

class Cooperator(Player):

    def __init__(self, dilemma: Dilemma, name: str = ""):
        """Cooperator"""
        super().__init__(dilemma,name)


    def strategy(self, opponent: Player) -> int:
        """Cooperates always"""
        return C


class Defector(Player):

    def __init__(self, dilemma: Dilemma, name: str = ""):
        """Defector"""
        super().__init__(dilemma,name)


    def strategy(self, opponent: Player) -> int:
        """Defects always"""
        return D


class Tft(Player):

    def __init__(self, dilemma: Dilemma, name: str = ""):
        """Tit-for-tat"""
        super().__init__(dilemma,name)


    def strategy(self, opponent: Player) -> int:
        """Cooperates first, then repeat last action of the opponent"""

        if self.trust:
            if D in opponent.history:
                self.trust = False
                return opponent.history[-1]
            return C
        else:
            return opponent.history[-1]


class Grudger(Player):

    def __init__(self, dilemma: Dilemma, name: str = ""):
        """Grudger"""
        super().__init__(dilemma,name)


    def strategy(self, opponent: Player) -> int:
        """
        Cooperates always, but if opponent ever defects, it will defect for the
        rest of the game
        """
        if self.trust:
            if D in opponent.history:
                self.trust = False
                return D
            return C
        else:
            return D


class Detective4MovsTft(Player):

    def __init__(self, dilemma: Dilemma, name: str = ""):
        """Four movement - tit for tat detective"""
        super().__init__(dilemma,name)
        self.first_movements=[C,D,C,C]


    def strategy(self, opponent: Player) -> int:
        """
        Starts with a fixed sequence of actions: [C,D,C,C]. After that, if
        the opponent has ever defected, plays 'TFT'. If not, plays 'Defector'.
        """
        if self.iteration <= 3:
            self.iteration+=1
            return self.first_movements[self.iteration-1]
        else:
            if D in opponent.history:
                return opponent.history[-1]
            else:
                return D

## **CLASE GAME**

In [22]:
class Game:

    def __init__(self, player_1: Player,
                       player_2: Player,
                       n_rounds: int = 100,
                       error: float = 0.0):
        """
        Game class to represent an iterative dilema

        Parameters:
            - player_1 (Player): first player of the game
            - player_2 (Player): second player of the game
            - n_rounds (int = 100): number of rounds in the game
            - error (float = 0.0): error probability (in base 1)
        """

        assert n_rounds > 0, "'n_rounds' should be greater than 0"

        self.player_1 = player_1
        self.player_2 = player_2
        self.n_rounds = n_rounds
        self.error = error

        self.score = (0.0, 0.0)  # this variable will store the final result of
                                 # the game, once the 'play()' function has
                                 # been called. The two values of the tuple
                                 # correspond to the points scored by the first
                                 # and second player, respectively.


    def play(self, do_print: bool = False) -> None:
        """
        Main call of the class. Play the game.
        Stores the final result in 'self.score'

        Parameters
            - do_print (bool = False): if True, should print the ongoing
            results at the end of each round (i.e. print round number, last
            actions of both players and ongoing score).
        """
        for iteration in range(self.n_rounds):
            action_1=self.player_1.strategy(self.player_2)
            action_2=self.player_2.strategy(self.player_1)
            action_1,bool_1 = self.change_action(action_1)
            action_2,bool_2= self.change_action(action_2)
            self.player_1.history.append(action_1)
            self.player_2.history.append(action_2)

            if do_print:
                print('-'*50)
                print(f'Play number {iteration+1}:')
                if bool_1:
                    print(f'{self.player_1.name} played:{action_1}. Action was changed due to error probability.')
                else:
                    print(f'{self.player_1.name} played:{action_1}. Action was not changed.')
                if bool_2:
                    print(f'{self.player_2.name} played:{action_2}. Action was changed due to error probability.')
                else:
                    print(f'{self.player_2.name} played:{action_2}. Action was not changed.')
                print(self.player_1.compute_scores(self.player_2))

        self.score = (self.player_1.compute_scores(self.player_2))
        

    def change_action(self, action):
        if random.random() < self.error:
            if action==C:
                return D, True
            else:
                return C, True
        else:
            return action, False

In [38]:
dilemma = Dilemma(2, -1, 3, 0)

cooperator_player = Cooperator(dilemma, "cooperator")
defector_player = Defector(dilemma, "defector")
tft_player = Tft(dilemma, "tft")
grudger_player = Grudger(dilemma, "grudger")
detective_player = Detective4MovsTft(dilemma, "detective")

#Modifica las siguientes líneas a conveniencia para llevar a cabo distintos tests
game = Game(tft_player, grudger_player, n_rounds=10, error=0)
game.play(do_print=True)
#print(game.score)

--------------------------------------------------
Play number 1:
tft played:0. Action was not changed.
grudger played:0. Action was not changed.
(2, 2)
--------------------------------------------------
Play number 2:
tft played:0. Action was not changed.
grudger played:0. Action was not changed.
(4, 4)
--------------------------------------------------
Play number 3:
tft played:0. Action was not changed.
grudger played:0. Action was not changed.
(6, 6)
--------------------------------------------------
Play number 4:
tft played:0. Action was not changed.
grudger played:0. Action was not changed.
(8, 8)
--------------------------------------------------
Play number 5:
tft played:0. Action was not changed.
grudger played:0. Action was not changed.
(10, 10)
--------------------------------------------------
Play number 6:
tft played:0. Action was not changed.
grudger played:0. Action was not changed.
(12, 12)
--------------------------------------------------
Play number 7:
tft played:0

## **CLASE TOURNAMENT**

In [43]:
class Tournament:

    def __init__(self, players: tuple[Player, ...],
                       n_rounds: int = 100,
                       error: float = 0.0,
                       repetitions: int = 2):
        """
        All-against-all tournament

        Parameters:
            - players (tuple[Player, ...]): tuple of players that will play the
         tournament
            - n_rounds (int = 100): number of rounds in the game
            - error (float = 0.0): error probability (in base 1)
            - repetitions (int = 2): number of games each player plays against
         the rest
        """

        self.players = players
        self.n_rounds = n_rounds
        self.error = error
        self.repetitions = repetitions

        # This is a key variable of the class. It is intended to store the
        # ongoing ranking of the tournament. It is a dictionary whose keys are
        # the players in the tournament, and its corresponding values are the
        # points obtained in their interactions with each other. In the end, to
        # see the winner, it will be enough to sort this dictionary by the
        # values.
        self.ranking = {player: 0.0 for player in self.players}  # initial vals


    def sort_ranking(self) -> None:
        self.ranking = dict(sorted(self.ranking.items(), key=lambda item: item[1],reverse=True))

    #pista: utiliza 'itertools.combinations' para hacer los cruces
    def play(self) -> None:
        """
        Main call of the class. It must simulate the championship and update
        the variable 'self.ranking' with the accumulated points obtained by
        each player in their interactions.
        """
        combinations = list(itertools.combinations(self.players, 2))
        for repetetion in range(self.repetitions):
            for combination in combinations:
                game=Game(combination[0],combination[1],self.n_rounds,self.error)
                print('*'*100)
                print(f'Nuevo enfrentamiento{combination[0].name,combination[1].name}')
                game.play(True)
                combination[0].clean_history()
                combination[1].clean_history()
                score_0,score_1=game.score
                print(score_0,score_1)
                
                self.ranking[combination[0]] = self.ranking[combination[0]]+ score_0
                self.ranking[combination[1]] = self.ranking[combination[1]] +score_1
            self.sort_ranking()


    def plot_results(self):
        """
        Plots a bar chart of the final ranking. On the x-axis should appear
        the names of the sorted ranking of players participating in the
        tournament. On the y-axis the points obtained.
        """
        raise NotImplementedError

In [44]:
dilemma = Dilemma(2, -1, 3, 0)

cooperator_player = Cooperator(dilemma, "cooperator")
defector_player = Defector(dilemma, "defector")
tft_player = Tft(dilemma, "tft")
grudger_player = Grudger(dilemma, "grudger")
detective_player = Detective4MovsTft(dilemma, "detective")
torneo=Tournament([cooperator_player,defector_player,tft_player,grudger_player,detective_player],10,0,1)
torneo.play()

****************************************************************************************************
Nuevo enfrentamiento('cooperator', 'defector')
--------------------------------------------------
Play number 1:
cooperator played:0. Action was not changed.
defector played:1. Action was not changed.
(-1, 3)
--------------------------------------------------
Play number 2:
cooperator played:0. Action was not changed.
defector played:1. Action was not changed.
(-2, 6)
--------------------------------------------------
Play number 3:
cooperator played:0. Action was not changed.
defector played:1. Action was not changed.
(-3, 9)
--------------------------------------------------
Play number 4:
cooperator played:0. Action was not changed.
defector played:1. Action was not changed.
(-4, 12)
--------------------------------------------------
Play number 5:
cooperator played:0. Action was not changed.
defector played:1. Action was not changed.
(-5, 15)
----------------------------------------

In [45]:
torneo.ranking

{<__main__.Tft at 0x7f56dff76d70>: 57.0,
 <__main__.Grudger at 0x7f56dff75330>: 46.0,
 <__main__.Defector at 0x7f56dff742e0>: 45.0,
 <__main__.Detective4MovsTft at 0x7f56dff769b0>: 45.0,
 <__main__.Cooperator at 0x7f56dff75cf0>: 29.0}

Para testear la implementación, por ejemplo prueba a reproducir el campeonato de la sección "3. One Tournament" del juego de Nicky Case. En dicho campeonato se enfrentaban las 5 estrategias básicas (cooperador, desertor, tft, grudger y detective) en un torneo con 10 rondas por interacción y sin error. Los resultados que debes obtener son los siguientes:
 - Tit-For-Tat: ganador con 57 puntos
 - Grudger: 46 puntos
 - Detective: 45 puntos
 - Desertor: 45 puntos
 - Cooperador: 29 puntos

In [ ]:
dilemma = Dilemma(2, -1, 3, 0)

cooperator_player = Cooperator(dilemma, "cooperator")
defector_player = Defector(dilemma, "defector")
tft_player = Tft(dilemma, "tft")
grudger_player = Grudger(dilemma, "grudger")
detective_player = Detective4MovsTft(dilemma, "detective")

all_players = (cooperator_player, defector_player, tft_player, grudger_player,
               detective_player)

tournament = Tournament(all_players, n_rounds=10, error=0.0, repetitions=1)
tournament.play()
tournament.plot_results()

## **CLASE EVOLUTION**

Implementa también la variante evolutiva del torneo, similar a la que se explica en la sección "4. Repeated Tournament" del juego de Nicky Case. Este módulo necesitará de inputs extra. Ten en cuenta lo siguiente:
 - La población inicial de jugadores no es directamente el input de jugadores que dé el usuario, sino que cada jugador tiene varios "individuos" o "réplicas" que jugarán su estrategia. Se propone dar al usuario dos opciones para definir esta población inicial:
   - Si el usuario define el tamaño de la población total (```int```), se asume que cada jugador comienza con el mismo número de representantes. Divide ese número entre el número de jugadores (redondeado al entero más próximo) y así obtendrás el número de individuos inicial de cada estrategia.
   - Si el usuario define una tupla de números de individuos (```list[int, ...]```), se asume que cada jugador tendrá el número de representantes que indique su índice dentro de dicha tupla.
 - Hay dos parámetros más que controlan el proceso evolutivo. 
   - En primer lugar, el porcentaje de individuos que se desea incluir en la selección natural tras cada ronda; esto es, el número de individuos *de la parte de abajo del ranking* que se van a eliminar y sustituir por los *individuos de la parte de arriba* (en caso de empate entre individuos, escoge al azar).
   - En segundo lugar, el número de generaciones que se van a simular. Una generación es básicamente un "Torneo de enfrentamiento directo" + un "Proceso de selección natural". 

Deberás tomar una decisión sobre qué hacer en la selección natural cuando hay *empates*. 

A continuación se incluye la plantilla de desarrollo sugerida.

In [ ]:
class Evolution:

    # Este método ya está implementado
    def __init__(self, players: tuple[Player, ...],
                       n_rounds: int = 100,
                       error: float = 0.0,
                       repetitions: int = 2,
                       generations: int = 100,
                       reproductivity: float = 0.05,
                       initial_population: tuple[int, ...] | int = 100):
        """
        Evolutionary tournament

        Parameters:
            - players (tuple[Player, ...]): tuple of players that will play the
         tournament
            - n_rounds (int = 100): number of rounds in each game
            - error (float = 0.0): error probability (in base 1)
            - repetitions (int = 2): number of games each player plays against
         the rest
            - generations (int = 100): number of generations to simulate
            - reproductivity (float = 0.05): ratio (base 1) of worst players
         that will be removed and substituted by the top ones in the natural
         selection process carried out at the end of each generation
            - initial_population (tuple[int, ...] | int = 100): list of
         individuals representing each players (same index as 'players' tuple)
         OR total population size (int).
        """

        self.players = players
        self.n_rounds = n_rounds
        self.error = error
        self.repetitions = repetitions
        self.generations = generations
        self.reproductivity = reproductivity

        if isinstance(initial_population, int):
            self.initial_population = [math.floor(initial_population
                                       / len(self.players))
                                       for _ in range(len(self.players))]
        else:
            self.initial_population = initial_population

        self.total_population = sum(self.initial_population)
        self.repr_int = int(self.total_population * self.reproductivity)

        self.ranking = {copy.deepcopy(player): 0.0 for i, player in
                        enumerate(self.players)
                        for _ in range(self.initial_population[i])}


    def natural_selection(self, result_tournament: dict[Player, float]) \
                          -> tuple[list,list]:
        """
        Kill the worst guys, reproduce the top ones. Takes the ranking once a
        face-to-face tournament has been played and returns another ranking,
        with the evolutionary changes applied

        Parameters:
            - result_tournament: the 'tournament.ranking' kind of dict.

        Results:
            - Same kind of dict ranking as the input, but with the evolutionary
         dynamics applied
        """
        raise NotImplementedError


    def count_strategies(self) -> dict[str, int]:
        """
        Counts the number of played alive of each strategy, based on the
        initial list of players. Should be computed analyzing the
        'self.ranking' variable. Useful for the results plot/print (not needed
        for the tournament itself)

        Results:
            - A dict, containing as values the name of the players and as
         values the number of individuals they have now alive in the tournament
        """
        raise NotImplementedError


    def play(self, do_print: bool = False):
        """
        Main call of the class. Performs the computations to simulate the
        evolutionary tournament.

        Parameters
            - do_print (bool = False): if True, should print the ongoing
         results at the end of each generation (i.e. print generation number,
         and number of individuals playing each strategy).
        """

        # HINT: Initialise the following variable
        #  > count_evolution = {player.name: [val] for player, val in
        #                       zip(self.players, self.initial_population)}
        # and use it to store the number of individuals each player retains at
        # the end of each generation, appending to its corresponding list value
        # the number of individuals each player has (obtained by calling
        # 'self.count_strategies()'). For example, at some point, it could have
        # the following value:
        # {'cooperator': [15, 10, 5, 0, 0, 0, 0, 0, 0, 0, 0],
        #  'defector': [5, 10, 15, 19, 14, 9, 4, 0, 0, 0, 0],
        #  'tft': [5, 5, 5, 6, 11, 16, 21, 25, 25, 25, 25]}

        raise NotImplementedError

    # Si quieres obtener un buen gráfico de la evolución, puedes usar este
    # método si has seguido la pista indicada en la cabecera del método
    # anterior. Ya está implementado, pero puede que necesites adaptarlo a tu
    # código.
    def stackplot(self, count_evolution: dict[str, list]) -> None:
        """
        Plots a 'stackplot' of the evolution of the tournament

        Parameters:
            - count_evolution (dict[Player, list]): a dictionary containing as
         keys the name of the strategies of the different players of the
         tournament. Each value is a list, where the 'i'-th position of that
         list indicates the number of individuals that player has at the end of
         the 'i'-th generation
         """

        COLORS = ['blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black']

        for i, name in enumerate(count_evolution.keys()):
            plt.plot([], [], label=name, color= COLORS[(i) % len(COLORS)])

        plt.stackplot(list(range(self.generations + 1)),
                      np.array(list(count_evolution.values())), colors=COLORS)

        plt.legend()
        plt.show()



Para testear el módulo anterior, puedes usar alguno de los experimentos mostrados en el juego de Nicky Case. Por ejemplo, un torneo con 5 TFT, 5 Desertores y 15 Cooperantes, sin error, con 10 rondas por interacción, con una reproductividad del 0.2, parece que en menos de 10 generaciones converge a una población dominada por TFT. Intenta replicar este resultado como se muestra a continuación:

In [ ]:
dilemma = Dilemma(2, -1, 3, 0)

cooperator_player = Cooperator(dilemma, "cooperator")
defector_player = Defector(dilemma, "defector")
tft_player = Tft(dilemma, "tft")

all_players = (cooperator_player, defector_player, tft_player)

evolution = Evolution(all_players, n_rounds=10, error=0.00, repetitions=1,
                      generations=10, reproductivity=0.2,
                      initial_population=(15, 5, 5))

evolution.play(True)

## **ESTRATEGIA PROFESORES**

In [ ]:
class Destructomatic(Player):

    def __init__(self, dilemma: Dilemma, name: str = ""):
        super().__init__(dilemma, name)
        self.defection_count = 0
        self.consecutive_cooperation = 0
        self.rounds_to_defect = 0

    def strategy(self, opponent: Player) -> int:
        """
        A gradual forgiveness strategy.
        Starts cooperating, increases defection periods for repeated defections,
        and resets after a series of cooperations.
        """

        turns = len(self.history)

        if turns == 0:  # First move, always cooperate
            return C

        if self.rounds_to_defect > 0:
            self.rounds_to_defect -= 1
            return D  # Defect if we're in a defection period

        last_move = opponent.history[-1]

        if last_move == C:
            self.consecutive_cooperation += 1
            if self.consecutive_cooperation >= 5:
                self.defection_count = 0
                self.consecutive_cooperation = 0
        else:  # Opponent defected
            self.defection_count += 1
            self.consecutive_cooperation = 0
            self.rounds_to_defect = self.defection_count
            return D  # Defect immediately after opponent's defection

        return C  # Cooperate otherwise

## **NUESTRA ESTRATEGIA**

In [ ]:
class NombreDeTuEstrategia(Player):

    def __init__(self, dilemma: Dilemma, name: str = ""):
        """NombreDeTuEstrategia"""
        super().__init__(dilemma,name)

        raise NotImplementedError

    def strategy(self, opponent: Player) -> int:
        """"""
        raise NotImplementedError

In [ ]:
# Haz todas las pruebas que necesites aquí.
# Por ejemplo:

game = Game(Destructomatic(dilemma, "destr"), Tft(dilemma, "tft"),
            dilemma, n_rounds=10, error=0.1)
game.play(do_print=True)

# O también:

dilemma = Dilemma(13, 0, 20, 4)
participants = (Destructomatic(dilemma, "destr"),
                Cooperator(dilemma, "coop1"),
                Defector(dilemma, "defect"),
                Cooperator(dilemma, "coop2"),
                Tft(dilemma, "tft"),
                Detective4MovsTft(dilemma, "detect"))

tournament = Tournament(participants, dilemma, n_rounds=100, error=0.01,
                        repetitions=2)
tournament.play()
tournament.plot_results()